<a href="https://colab.research.google.com/github/mystlee/2024_CSU_AI/blob/main/chapter5/torch_rnn_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 모델 구성, 학습 등을 위한 라이브러리 import   
### torch   
  - pytorch 프레임워크   

### torch.nn
  - nn = neural network   
  - 딥러닝 관련 라이브러리  
  - fully-connected layer, conv layer 등을 포함  

### torch.nn.functional   
  - 활성화 함수와 같은 딥러닝 관련 함수 라이브러리   
  - Softmax, ReLU 함수와 같은 활성화 함수 등등 포함

### torch.optim
  - 모델 학습을 위한 옵티마이저 라이브러리   
  - SGD, AdaGrad, RMSProp, Adam 등 옵티마이저 포함

### datasets
  - Huggingface에서 제공하는 dataset 다운로드 및 관리 패키지   
  

### transformers
  - Huggingface에서 제공하는 transformer관련 패키지   
  - 기본적으로 transformer에 사용되는 다양한 모듈을 포함하고 있지만, 예시에서는 tokenzier활용을 위한 용도


In [1]:
!pip install datasets sentencepiece transformers

import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## 1-1. 데이터셋 변환   
모델 입력에 맞게 데이터셋의 format을 변환

In [2]:
# TED Talks 데이터셋 (포루투칼어 <-> 영어) 로드
raw_datasets = load_dataset('ted_hrlr', 'pt_to_en')

# 학습(training), 검증(validation), 테스트(test) 데이터셋 분할
train_data = raw_datasets['train']
valid_data = raw_datasets['validation']
test_data = raw_datasets['test']

print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(valid_data)}")
print(f"Test samples: {len(test_data)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

ted_hrlr.py:   0%|          | 0.00/6.68k [00:00<?, ?B/s]

The repository for ted_hrlr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ted_hrlr.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/51786 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1194 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1804 [00:00<?, ? examples/s]

Train samples: 51786
Validation samples: 1194
Test samples: 1804


## 1-2. 데이터셋 변환 (텍스트 tokenizing)   
Tokenizer를 이용해서 텍스트를 숫자 (index)로 변환   
BERT (transformer의 일종)에 사용된 tokenizer를 활용   
 - BOS (\<s\>): begin of sentence -> 문장의 시작   
 - EOS (\<\s\>): emd of sentence -> 문장의 끝   
 - PAD: 문장의 공백을 채울 때 사용


In [3]:
# 토크나이저 (tokenizer) 설정 (BERT의 기본 토크나이저 사용)
tokenizer_src = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer_tgt = AutoTokenizer.from_pretrained('bert-base-uncased')

# special token 설정
bos_token = '[BOS]'
eos_token = '[EOS]'
pad_token = '[PAD]'

# special token을 tokenizer의 목록에 추가
special_tokens = {'bos_token': bos_token,
                  'eos_token': eos_token,
                  'pad_token': pad_token}

tokenizer_src.add_special_tokens(special_tokens)
tokenizer_tgt.add_special_tokens(special_tokens)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

2

## 1-3. 데이터셋 변환 (텍스트 data loader)   
입력 문장과 출력 문장의 포맷을 맞춤   
 - 입력 문장: 문장 앞에 BOS 토큰을 삽입   
 - 출력 문장: 문장 뒤에 EOS 토큰을 삽입   
또 하나의 special token
 - 번역 및 NLP에서는 입/출력 domain을 표기해주기 위한 특수한 토큰이 사용됨
 - 현재 예시에서는 번역 (입력: 포루투칼어, 출력: 영어)   
기타
 - 문장의 최대 길이: max_length   
 - 문장 자르기 옵션: truncation   
 - 패딩: padding   

In [4]:
from torch.utils.data import DataLoader

# 데이터 전처리 함수
def preprocess_function(examples):
    inputs = [bos_token + ' ' + ex['pt'] + ' ' + eos_token
              for ex in examples['translation']]
    targets = [bos_token + ' ' + ex['en'] + ' ' + eos_token
               for ex in examples['translation']]

    model_inputs = tokenizer_src(inputs, max_length = 128,
                                 truncation = True, padding = 'max_length')

    labels = tokenizer_tgt(targets, max_length = 128,
                           truncation = True, padding = 'max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 전처리
train_dataset = train_data.map(preprocess_function, batched = True,
                               remove_columns = train_data.column_names)
valid_dataset = valid_data.map(preprocess_function, batched = True,
                               remove_columns = valid_data.column_names)
test_dataset = test_data.map(preprocess_function, batched = True,
                             remove_columns=test_data.column_names)

def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch], dtype = torch.long)
    labels = torch.tensor([item['labels'] for item in batch], dtype = torch.long)
    return {'input_ids': input_ids, 'labels': labels}

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size = batch_size,
                              shuffle = True, collate_fn = collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size = batch_size,
                              collate_fn = collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size,
                             collate_fn = collate_fn)


Map:   0%|          | 0/51786 [00:00<?, ? examples/s]

Map:   0%|          | 0/1194 [00:00<?, ? examples/s]

Map:   0%|          | 0/1804 [00:00<?, ? examples/s]

## 2. 모델 구조   
모델 구조 작성   
- 일반적으로 class 의 \_\_init\_\_ 함수부분에서 모듈들은 선언하고,
- 그 다음 forward 함수에 전체적인 흐름 작성   
- Encoder-decoder 구조로 가장 popular하게 쓰이는 구조# 2. 모델 구조   
모델 구조 작성   
- 일반적으로 class 의 \_\_init\_\_ 함수부분에서 모듈들은 선언하고,
- 그 다음 forward 함수에 전체적인 흐름 작성   
- Encoder-decoder 구조로 가장 popular하게 쓰이는 구조

<img src = "https://nkw011.github.io/assets/image/seq_to_seq/seq2.png" width = "80%" height = "70%">   

출처: <https://nkw011.github.io/nlp/seqtoseq/>   

In [5]:
# 모델 정의
class Seq2SeqRNN(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim,
                 embed_dim,
                 hidden_dim,
                 padding_idx):

        super(Seq2SeqRNN, self).__init__()
        self.encoder = nn.Embedding(input_dim, embed_dim,
                                    padding_idx = padding_idx)
        self.decoder = nn.Embedding(output_dim, embed_dim,
                                    padding_idx = padding_idx)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, src, tgt):
        # Encoder
        embedded_src = self.encoder(src)
        _, hidden = self.rnn(embedded_src)
        # Decoder
        embedded_tgt = self.decoder(tgt)
        outputs, _ = self.rnn(embedded_tgt, hidden)
        outputs = self.fc(outputs)
        return outputs


## 3-1. 학습 함수, 손실 함수 및 옵티마이저 정의   
학습 함수   
- batch 단위로 update 진행
- error와 gradients를 계산하고 업데이트!
- classification task -> cross entropy loss를 사용!
 - token을 분류해서 맞추는 의미
 - cross entorpy를 측정할 때, padding_idx라는 변수가 사용되는데, 이는 loss를 계산할 때, padding되는 부분에 대해서 계산하지 않기 위함

- optimizer는 Adam 사용
 - [pytorch optimizer] <https://pytorch.org/docs/stable/optim.html>
 - SGD, RMSprop 등 다양한 optimizer 활용 가능


Loss와 optimizer를 이용해서 모델 학습!



In [6]:
input_dim = len(tokenizer_src)
output_dim = len(tokenizer_tgt)
embed_dim = 256
hidden_dim = 512
padding_idx = tokenizer_src.pad_token_id

# 모델 객체 생성
model = Seq2SeqRNN(input_dim, output_dim, embed_dim, hidden_dim, padding_idx)

# 손실함수 및 optimizer 설정
criterion = nn.CrossEntropyLoss(ignore_index = padding_idx)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

# 학습 함수
def train(model, dataloader, optimizer, criterion, epoch, log_interval = 100):
    model.train()
    total_loss = 0
    epoch_loss = 0  # 에포크 전체 손실 누적 변수 추가
    total_steps = 0  # 전체 스텝 수
    for batch_idx, batch in enumerate(dataloader):
        src = batch['input_ids']
        tgt = batch['labels']

        optimizer.zero_grad()
        output = model(src, tgt[:, :-1])  # 마지막 token 제외
        output = output.reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)  # 첫 token 토큰 제외

        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        loss_value = loss.item()
        total_loss += loss_value
        epoch_loss += loss_value
        total_steps += 1

        if total_steps % log_interval == 0:
            avg_loss = total_loss / log_interval
            print(f'Epoch [{epoch+1}], Step [{total_steps}], Loss: {avg_loss:.4f}')
            total_loss = 0

    avg_epoch_loss = epoch_loss / total_steps
    print(f'Epoch {epoch+1} Completed. Average Loss: {avg_epoch_loss:.4f}')
    return avg_epoch_loss



num_epochs = 5
for epoch in range(num_epochs):
    loss = train(model, train_dataloader, optimizer, criterion, epoch, log_interval = 10)

Epoch [1], Step [10], Loss: 8.5808
Epoch [1], Step [20], Loss: 5.8317
Epoch [1], Step [30], Loss: 5.7096
Epoch [1], Step [40], Loss: 5.4407
Epoch [1], Step [50], Loss: 5.3703
Epoch [1], Step [60], Loss: 5.2811
Epoch [1], Step [70], Loss: 5.1431
Epoch [1], Step [80], Loss: 5.1115
Epoch [1], Step [90], Loss: 5.0778
Epoch [1], Step [100], Loss: 4.9983
Epoch [1], Step [110], Loss: 4.9607
Epoch [1], Step [120], Loss: 4.9783
Epoch [1], Step [130], Loss: 4.8616
Epoch [1], Step [140], Loss: 4.8421
Epoch [1], Step [150], Loss: 4.8857
Epoch [1], Step [160], Loss: 4.8018
Epoch [1], Step [170], Loss: 4.8083
Epoch [1], Step [180], Loss: 4.7153
Epoch [1], Step [190], Loss: 4.6617
Epoch [1], Step [200], Loss: 4.7396
Epoch [1], Step [210], Loss: 4.7644
Epoch [1], Step [220], Loss: 4.7052
Epoch [1], Step [230], Loss: 4.6459
Epoch [1], Step [240], Loss: 4.6082
Epoch [1], Step [250], Loss: 4.6792
Epoch [1], Step [260], Loss: 4.6498
Epoch [1], Step [270], Loss: 4.6289
Epoch [1], Step [280], Loss: 4.6199
E

KeyboardInterrupt: 

In [8]:
def translate(model, sentence):
    model.eval()
    with torch.no_grad():
        src = bos_token + ' ' + sentence + ' ' + eos_token
        src_tokenized = tokenizer_src(src, return_tensors = 'pt',
                                      max_length = 128,
                                      truncation = True,
                                      padding = 'max_length')
        src_input_ids = src_tokenized['input_ids'] # shape: [1, src_seq_len]

        # Encoder에서 컨텍스트 벡터 생성
        embedded_src = model.encoder(src_input_ids)
        _, hidden = model.rnn(embedded_src)

        # Decoder에서 초기 입력 설정 (<BOS> 토큰)
        tgt_input = torch.tensor([[tokenizer_tgt.bos_token_id]],
                                 dtype = torch.long)
        translated_tokens = []

        for _ in range(50):  # 최대 생성 길이 설정
            # shape: [1, seq_len, embed_dim]
            embedded_tgt = model.decoder(tgt_input)
            # output shape: [1, seq_len, hidden_dim]
            output, hidden = model.rnn(embedded_tgt, hidden)
            # 마지막 time step의 출력, shape: [1, output_dim]
            output = model.fc(output[:, -1, :])
            pred_token = output.argmax(1)  # shape: [1]

            translated_tokens.append(pred_token.item())
            if pred_token.item() == tokenizer_tgt.eos_token_id:
                break

            # pred_token의 dim.을 [1, 1]로
            pred_token = pred_token.unsqueeze(1)  # shape: [1, 1]
            # tgt_input (앞서 예측된 tokens) 과 pred_token을 연결
            tgt_input = torch.cat([tgt_input, pred_token], dim=1)  # shape: [1, seq_len + 1]

        translated_sentence = tokenizer_tgt.decode(translated_tokens, skip_special_tokens=True)
        return translated_sentence


# 예시 문장 번역
sample_sentence = "Olá, como você está?"
translation = translate(model, sample_sentence)
print(f'번역 결과: {translation}')


번역 결과: he was a very difficult battle for the first time.
